# ResNet Implementation

Kurz-Erklärung:

- ResNet = Regelmäßige Layer-Skips: Ein Convolutional Encoder-Decoder (bzw Bottleneck) Block erhält als Eingabe die addierten Ausgaben der beiden vorherigen Blöcke.

- ResNeXt = ResNet aber mit mehreren separaten Streams: Die Eingabe wird intern mit mehreren kleineren Layer parallel verarbeitet und später für den Layer-Skip zusammengeführt.

![ResNet & ResNeXt](resnext_block.png)

Grafik von Xie et al, "Aggregated Residual Transformations for Deep Neural Networks", 2017, arXiv:1611.05431v2

In [ ]:
!pip install pandas numpy sklearn matplotlib
!conda install tensorflow-gpu

TODO random state setzen? <https://stackoverflow.com/a/52897289>

## Daten

Die Daten bestehen aus 28x28 Bilder in einer CSV-Datei. Die erste Spalte enthält das Label und alle anderen Spalten sind die Helligkeit des jeweiligen Pixels als Byte.

In [1]:
import pandas as pd
X_train = pd.read_csv("data/sign_mnist_train.csv")
X_test = pd.read_csv("data/sign_mnist_test.csv")

# separate label column
y_train = X_train.pop('label')
y_test = X_test.pop('label')

# values are 0-255 right now, scale them to 0.0-1.0 instead
X_train = X_train.divide(255)
X_test = X_test.divide(255)

# rearrange from 1d to 3d (w,h,c)
# TODO are the additional "color" channels (np.tile) necessary?
import numpy as np
X_train = np.tile(np.expand_dims(X_train.to_numpy().reshape(-1,28,28),axis=3),3)
X_test = np.tile(np.expand_dims(X_test.to_numpy().reshape(-1,28,28),axis=3),3)

# label is category, convert to one-hot vector
from sklearn.preprocessing import LabelBinarizer
label_binarizer = LabelBinarizer()
label_binarizer.fit(y_train)
label_binarizer.fit(y_test)
y_train = label_binarizer.transform(y_train)
y_test = label_binarizer.transform(y_test)
labels_amount = len(y_train[0])

# imagenet weights want 1000 classes
y_train = np.pad(y_train, ((0,0),(0,1000-labels_amount)))
y_test = np.pad(y_test, ((0,0),(0,1000-labels_amount)))

print(f"Amount of samples: Train {len(y_train)} vs. Test {len(y_test)}")
print(f'Amount of distinct labels: {labels_amount}')

Amount of samples: Train 27455 vs. Test 7172
Amount of distinct labels: 24


**TODO should we scale images to (224, 224, 3) somehow?** (without actually resizing them individually, which has proven as a surefire method of frying the cpu)

<https://keras.io/api/applications/resnet/#resnet50v2-function>

In [32]:
#import imutils
#import numpy as np

#X_train_big = []
#for row in np.tile(np.expand_dims(X_train.to_numpy(),axis=2),3):
#    X_train_big.append( imutils.resize(row.reshape(28,28,3),height=224,width=224) )

## Training

ResNet wird mittlerweile von Keras als fertige Architektur angeboten.

In [2]:
from tensorflow.keras.applications import ResNet50V2

model = ResNet50V2()  # weights=None,classes=labels_amount)

# https://keras.rstudio.com/reference/compile.html
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [11]:
model.summary()

Model: "resnet50v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
_________________________________________________________________________________________

Damit nicht jedes mal neu trainiert werden muss, können stattdessen an dieser Stelle Gewichte geladen werden.

In [3]:
model.load_weights('data/resnet_weights.h5')

Das Training kann ruhig im Hintergrund laufen. Es sind zwar nur wenige Epochen, die dauern aber dennoch mehrere Stunden.

In [ ]:
history = model.fit(X_train, y_train, epochs = 10, validation_split=0.2)

Die Gewichte werden anschließend gespeichert, um wie zuvor erwähnt wiederverwendet werden zu können.

In [27]:
model.save_weights('data/resnet_weights.h5')

## Überprüfen

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='Training')
plt.plot(history.history['val_loss'], label='Validation')
plt.title('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

plt.plot(history.history['categorical_accuracy'], label='Training')
plt.plot(history.history['val_categorical_accuracy'], label='Validation')
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.show()

In [25]:
model.evaluate(X_test, y_test)

225/225 [==============================] - 118s 522ms/step - loss: 2.1010 - accuracy: 0.5474


[2.1009738445281982, 0.5474065542221069]

## Conclusio

Es dauert ziemlich lange, ResNet zu trainieren. Die Trainingszeit kann durch Reduktion der Trainingsdaten verkürzt werden, allerdings besteht dann die Gefahr des Overfittings.